# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(data):
    
    y = data[:, -1]
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from pprint import pprint
import random
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('Iris 2.csv', index_col = 'Id')

train_df, test_df = train_test_split(df, train_size=100, random_state=42)
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica


In [ ]:
data = train_df.values


array(['Iris-versicolor', 'Iris-virginica', 'Iris-versicolor',
       'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica', 'Iris-setosa',
       'Iris-virginica', 'Iris-virginica', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor',
       'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa',
       'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor',
       'Iris-virginica', 'Iris-virginica', 'Iris-versicolor',
       'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica',
       'Iris-setosa', 'Iris-setosa', 'I

In [ ]:
def check_purity(data): # возвращает True, если в разбиение все данные относятся к одному классу

    y = data[:, -1]
    unique_classes = np.unique(y)

    if len(unique_classes) == 1:
        
        return True
    
    else:

        return False

In [ ]:
def classify_data(data): # возвращает класс, которому принадлежат данные 

    y = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(y, return_counts = True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]

    return classification

In [ ]:
def get_potential_splits(data): # возвращает словарь, где ключ - это номер колонки, а значени это массив из вариантов возможных и уникальных значений для разбиния
    
    potential_splits = {}
    n_columns = data.shape[1]

    for column_index in range(n_columns - 1): # отбросили y

        values = data[:, column_index]
        unique_values = np.unique(values)

        potential_splits[column_index] = unique_values
    
    return potential_splits

In [ ]:
def split_data(data, split_column, split_value): # разделяет массив значений на 2, которые возвращают значения больше и меньше заданного для разбиения

    split_column_values = data[:, split_column] 

    data_below = data[split_column_values <= split_value]
    
    data_above = data[split_column_values > split_value]

    return data_below, data_above

In [ ]:
def all_entropy(data, data_below, data_above): # считаем общую энтропию

    n_data_points = len(data)

    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points

    all_entropy = (p_data_below * entropy(data_below) + p_data_above * entropy(data_above))

    return all_entropy

In [ ]:
def determine_best_split(data, potential_splits): # возвращает лучшее значение для разделения, а также название фичи
    
    overall_entropy = 10000

    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = all_entropy(data, data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [ ]:
def decision_tree_algorithm(df, counter=0, min_samples=1, max_depth=100):
    
    # обработка данных
    if counter == 0: 

        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values

    else:

        data = df           
    
    
    # остановливает рекурсию, а также возвращает нужный класс для заданных данных

    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        
        classification = classify_data(data)
        
        return classification

    else:

        counter += 1
 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        

        feature_name = COLUMN_HEADERS[split_column]
        
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
       ##  проверка на то, чтобы у sub_tree были разные значения для ответа

        if yes_answer == no_answer:

            sub_tree = yes_answer

        else:

            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [ ]:
def predict(test, tree):

    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    if test[feature_name] <= float(value):

        answer = tree[question][0]

    else:

        answer = tree[question][1]

    if not isinstance(answer, dict): # если ответ не словарь, то возвращаем ответ
    
        return answer
    
    else:

        next_tree = answer

        return predict(test, next_tree)

In [ ]:
def decision_tree_predictions(df, tree):

    predictions = df.apply(predict, args=(tree,), axis=1)
    
    return predictions

In [ ]:
def accuracy_DR(df, tree):

    df["classification"] = decision_tree_predictions(df, tree)
    df["classification_correct"] = df["classification"] == df["Species"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [ ]:
tree_purity = decision_tree_algorithm(train_df)
pprint(tree_purity)

{'PetalWidthCm <= 0.6': ['Iris-setosa',
                         {'PetalWidthCm <= 1.7': [{'PetalLengthCm <= 5.1': [{'PetalWidthCm <= 1.4': ['Iris-versicolor',
                                                                                                     {'SepalWidthCm <= 2.5': [{'SepalLengthCm <= 6.0': ['Iris-virginica',
                                                                                                                                                        'Iris-versicolor']},
                                                                                                                              'Iris-versicolor']}]},
                                                                            'Iris-virginica']},
                                                  {'PetalLengthCm <= 4.8': [{'SepalWidthCm <= 3.0': ['Iris-virginica',
                                                                                                     'Iris-versicolor']},
           

In [ ]:
tree_depth = decision_tree_algorithm(train_df, max_depth = 5)
pprint(tree_depth)

{'PetalWidthCm <= 0.6': ['Iris-setosa',
                         {'PetalWidthCm <= 1.7': [{'PetalLengthCm <= 5.1': [{'PetalWidthCm <= 1.4': ['Iris-versicolor',
                                                                                                     {'SepalWidthCm <= 2.5': ['Iris-virginica',
                                                                                                                              'Iris-versicolor']}]},
                                                                            'Iris-virginica']},
                                                  {'PetalLengthCm <= 4.8': [{'SepalWidthCm <= 3.0': ['Iris-virginica',
                                                                                                     'Iris-versicolor']},
                                                                            'Iris-virginica']}]}]}


In [ ]:
tree_samples = decision_tree_algorithm(train_df, min_samples = 5)
pprint(tree_samples)

{'PetalWidthCm <= 0.6': ['Iris-setosa',
                         {'PetalWidthCm <= 1.7': [{'PetalLengthCm <= 5.1': [{'PetalWidthCm <= 1.4': ['Iris-versicolor',
                                                                                                     {'SepalWidthCm <= 2.5': ['Iris-virginica',
                                                                                                                              'Iris-versicolor']}]},
                                                                            'Iris-virginica']},
                                                  'Iris-virginica']}]}


In [ ]:
accuracy_p = accuracy_DR(test_df, tree_purity)
print('Accuracy for purity: '+ str(accuracy_p))

accuracy_d = accuracy_DR(test_df, tree_depth)
print('Accuracy for depth: ' + str(accuracy_d))

accuracy_s = accuracy_DR(test_df, tree_samples)
print('Accuracy for samples: ' + str(accuracy_s))

Accuracy for purity: 0.98
Accuracy for depth: 0.96
Accuracy for samples: 0.96


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv('churn.csv')
y = df['Exited']

In [ ]:
y = df['Exited']

In [ ]:
drop_col = ['Surname','CustomerId','RowNumber', 'Exited']
df_new = df.drop(drop_col, 1)

In [ ]:
# обработка категориальных данных
merged_df = df_new.drop(columns=['Gender','Geography']) 
enc = OneHotEncoder(handle_unknown='ignore')

encoded_df = enc.fit_transform(df_new[['Gender','Geography']])
column_name = enc.get_feature_names(['Gender','Geography'])

encoded_df_col = pd.DataFrame(encoded_df.toarray(), columns= column_name)
merged_df = merged_df.join(encoded_df_col)

In [ ]:
merged_df['Exited'] = y

In [ ]:
merged_df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain,Exited
0,619,42,2,0.00,1,1,1,101348.88,1.0,0.0,1.0,0.0,0.0,1
1,608,41,1,83807.86,1,0,1,112542.58,1.0,0.0,0.0,0.0,1.0,0
2,502,42,8,159660.80,3,1,0,113931.57,1.0,0.0,1.0,0.0,0.0,1
3,699,39,1,0.00,2,0,0,93826.63,1.0,0.0,1.0,0.0,0.0,0
4,850,43,2,125510.82,1,1,1,79084.10,1.0,0.0,0.0,0.0,1.0,0


In [ ]:
train_df, test_df = train_test_split(merged_df, train_size=0.8, random_state=42)

In [ ]:
def get_potential_splits(data, random_subspace): # исходная функция + random_subspace для выборки данных в RandomForest
    
    potential_splits = {}
    _, n_columns = data.shape
    column_indices = list(range(n_columns - 1))    
    
    if random_subspace and random_subspace <= len(column_indices):
        column_indexs = random.sample(population=column_indices, k=random_subspace)
    
    for column_index in column_indexs:          
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [ ]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5, random_subspace=None): # исходная функция + random_subspace для выборки данных в RandomForest
    
  
    if counter == 0:
        
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values

    else:

        data = df           
    
    
    
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
   
    else: 
         
        counter += 1

        potential_splits = get_potential_splits(data, random_subspace)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        feature_name = COLUMN_HEADERS[split_column]
        
        
        question = "{} <= {}".format(feature_name, split_value)
      
        sub_tree = {question: []}
        
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth, random_subspace)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth, random_subspace)
        
      
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [ ]:
def bootstrapping(train_df, n_bootstrap):
  
    bootstrap_indexes = np.random.randint(low=0, high=len(train_df), size=n_bootstrap)
    df_bootstrapped = train_df.iloc[bootstrap_indexes]
    
    return df_bootstrapped

In [ ]:
def random_forest_algorithm(train_df, n_trees, n_bootstrap, n_features, dt_max_depth): # создаем лес, который состоит из decision_tree

    forest = []

    for i in range(n_trees):
      
        df_bootstrapped = bootstrapping(train_df, n_bootstrap)
        tree = decision_tree_algorithm(df_bootstrapped, max_depth=dt_max_depth, random_subspace=n_features)
        forest.append(tree)
    
    return forest

In [ ]:
def random_forest_predictions(test_df, forest):
    df_predictions = {}

    for i in range(len(forest)):

        column_name = "tree_{}".format(i)
        predictions = decision_tree_predictions(test_df, tree=forest[i]) # смотрим, что предсказало каждое дерево
        df_predictions[column_name] = predictions

    df_predictions = pd.DataFrame(df_predictions)
    random_forest_predictions = df_predictions.mode(axis=1)[0] # выбираем самый частый класс который встречается в строке
    
    return random_forest_predictions

In [ ]:
forest = random_forest_algorithm(train_df, n_trees=21, n_bootstrap=4000, n_features=7, dt_max_depth=10)
predictions = random_forest_predictions(test_df, forest)

In [ ]:
accuracy = accuracy_score(predictions, test_df["Exited"])

print("Accuracy RF: " + str(accuracy))

Accuracy RF: 0.8665


In [ ]:
pprint(forest[0]) # одно из деревьев 

{'Age <= 41.0': [{'Gender_Male <= 0.0': [{'IsActiveMember <= 0.0': [{'NumOfProducts <= 1.0': [{'Balance <= 49572.73': [{'CreditScore <= 461.0': [0.0,
                                                                                                                                                 {'Geography_Spain <= 0.0': [{'Age <= 35.0': [{'Balance <= 35741.69': [0.0,
                                                                                                                                                                                                                       1.0]},
                                                                                                                                                                                              {'Age <= 40.0': [{'Age <= 36.0': [0.0,
                                                                                                                                                                                   